Rafał Nowak
# Numerical Optimization
## Problem set 3

**Problem 3.1** (3 pts)

In this problem we consider univariate function $f:\mathbb R\to\mathbb R$.
Let us remind the idea of bracketing method
> _Bracketing_ is the process of identifying an interval in which a local minimum lies and then successively shrinking the interval.

Implement the method `(a,b) = find_initial_bracket(f)` which for given function $f$ gives the bracketing interval $(a,b)$ such that there exist local minimum $c\in(a,b)$ satisfying $f(a)>f(c)<f(b)$.

In [1]:
# Solution
function find_initial_bracket(f, x=0; s=1e-2, k=2.0)
    a, ya = x, f(x)
    b, yb = a + s, f(a + s)
    if yb > ya
        a, b = b, a
        ya, yb = yb, ya
        s = -s
    end
    while true
        c, yc = b + s, f(b + s)
        if yc > yb
            return a < c ? (a, c) : (c, a)
        end
    a, ya, b, yb = b, yb, c, yc
    s *= k
    end
end

find_initial_bracket (generic function with 2 methods)

In [16]:
# Example test
f(x) = 3*x^2 + exp(.3*x-9) + 20*x - 20
a, b = find_initial_bracket(f)
@show a, b

(a, b) = (-5.109999999999999, -1.27)


(-5.109999999999999, -1.27)

```julia
using Plotly
plot( f, a , b )
```
<img src="f_bracket.png">

**Problem 3.2** (4 pts)

In this problem we consider [unimodal function](https://www.wikiwand.com/en/Unimodality#/Unimodal_function)
and will play with _golden section search_ method.

First you should implement the _Fibonacci search_ algorithm provided that you have the (global) array of Fibonacci numbers. Next you should implement the _golden section search_ which uses only _golden ratio_ $\varphi = (1+\sqrt 5)/2$.

* Implement the [Fibonacci search algorithm](https://www.wikiwand.com/en/Golden-section_search#/Fibonacci_search)<br/>`(a, b) = fibonacci_search(f, a, b, n; ε=1e-4)`<br/>to be run on univariate function $f$, with bracketing interval $[a, b]$, for $n > 1$ function evaluations. It should return the new interval $(a, b)$. The optional parameter $\varepsilon$ should control the lowest-level interval length.
* Implement [Golden section search](https://www.wikiwand.com/en/Golden-section_search#)<br/>`(a, b) = gs_search(f, a, b, n)`<br/> to be run on a univariate function $f$ , with bracketing interval $[a, b]$ , for $n > 1$ function evaluations. It returns the new interval $(a, b)$. Guaranteeing convergence to within $\varepsilon$ requires $n = (b-a)/(\varepsilon \ln\varphi)$.

Present the results on various kind of functions.

References:
- [Fibonacci Search in Optimization of Unimodal Functions](https://www.maplesoft.com/applications/view.aspx?SID=4193&view=html)
- [Golden section search](https://www.wikiwand.com/en/Golden-section_search#)

---

Frow now we assume that our black box function has additional parameter `order`.
The meaning is as follows
```julia
function my_func(x; order=0)
    if order==0
        return value
    elseif order==1
        return value, gradient
    elseif order==2
        return value, gradient, hessian
    else
        # raise an  error
    end
end
```

**Problem 3.3.** (5 pts)

Observe that the previous methods require only to evaluate the value of the objective function $f$.
From now we assume we know also its derivative.<br/>
For example, consider the function
$$ f(x) = x^4 + 16x^2 + 18(x-4) e^x\qquad (x\in\mathbb R). $$
First implement the function
```julia
# function gets scalar x as input and returns f(x) and f'(x)
function my_func(x, order=0)
    value = ____
    if order == 0
        return value
    elseif order == 1
        gradient = ____
        return (value, gradient)
    end
end
```
Do not forget to experiment with other (nice) functions.

* Implement bisection method to find the minimum of $f$
* Initialize the method with $x_{\text{left}}=-10$ and $x_{\text{right}}=10$.

```julia
# bisection gets function f, performs bisection search
# on interval [a, b] and returns an eps-suboptimal
# solution x; i.e. f(x)-f(x^*) <= eps .
function bisection(fun, MIN, MAX; epsilon=1e-5, max_iter=65536)
    # counting the number of iterations
    counter = 0
    while true
        counter +=1
        MID = ( MAX + MIN ) / 2

        # Oracle access to the function value and gradient
        value, gradient = fun( MID, order=1 )

        # provide an upper bound for the suboptimality of MID in terms of
        # the magnitude of the gradient and distance from the optimum
        ###############################
        # TODO: suboptimality = ???
        ###############################

        if suboptimality <= eps
            break
        end

        if gradient > 0
          ###############################
          # TODO: Updating the interval #
          ###############################
        else
          ###############################
          # TODO: Updating the interval #
          ###############################
        end
    end
    @printf( "Number of Iterations: %d", counter )
    @printf( "Suboptimal point: %1.15"', MID )
    @printf( "Suboptimal value: %1.15"', value )
    return MID    
end
```

Test your method with the function $f$ and the following functions (written in Python)
```python

###############################################################################
def boyd_example_func(x, order=0):
  a=np.matrix('1  3')
  b=np.matrix('1  -3')
  c=np.matrix('-1  0')
  x=np.asmatrix(x)
  
  value = exp(a*x-0.1)+exp(b*x-0.1)+exp(c*x-0.1)
  if order==0:
      return value
  elif order==1:
      gradient = a.T*exp(a*x-0.1)+b.T*exp(b*x-0.1)+c.T*exp(c*x-0.1)
      return (value, gradient)
  elif order==2:
      gradient = a.T*exp(a*x-0.1)+b.T*exp(b*x-0.1)+c.T*exp(c*x-0.1)
      hessian = a.T*a*exp(a*x-0.1)+b.T*b*exp(b*x-0.1)+c.T*c*exp(c*x-0.1)
      return (value, gradient, hessian)
  else:
        raise ValueError("The argument \"order\" should be 0, 1 or 2")


###############################################################################
def quadratic( H, b, x, order=0 ):
    """ 
    Quadratic Objective
    H:          the Hessian matrix
    b:          the vector of linear coefficients
    x:          the current iterate
    order:      the order of the oracle. For example, order=1 returns the value of the function and its gradient while order=2 will also return the hessian
    """
    H = np.asmatrix(H)
    b = np.asmatrix(b)
    x = np.asmatrix(x)
    
    value = 0.5 * x.T * H * x + b.T * x

    if order == 0:
        return value
    elif order == 1:
        gradient = H * x + b
        return (value, gradient)
    elif order == 2:
        gradient = H * x + b
        hessian = H
        return (value, gradient, hessian)
    else:
        raise ValueError("The argument \"order\" should be 0, 1 or 2")
```

---

**Problem 3.4** (3 pts)

Implement the *line_search* algorithm, which general idea is as follow:
```julia
function line_search(f, x, d)
    objective = α -> f(x + α*d)
    a, b = bracket_minimum(objective)
    α = minimize(objective, a, b)
    return x + α*d
end
```

In this problem you should implement the function
```julia
function exact_line_search( f, x, direction, eps=1e-9, maximum_iterations=65536 )
    # TODO
    
end
```

where the *bisection* method is used for optimizing in `line_search` and
* `f` is the function to optimize; assume one can call it like `value, gradient = f( x, order=1 )`
* `x` is the the current iterate
* `direction` is the direction along which to perform the linesearch
* `eps` is the maximum allowed error in the resulting stepsize $t$
* `maximum_iterations` is the maximum allowed number of iterations

**Problem 3.5.** (3 pts)

In this problem you should implement backtracking linesearch algorithm
```julia
function backtracking_line_search( f, x, direction, α=0.4, β=0.9, maximum_iterations=65536 )
    # TODO
    
end
```

where the *bisection* method is used for optimizing in `line_search` and
* `f` is the function to optimize; assume one can call it like `value, gradient = f( x, order=1 )`
* `x` is the the current iterate
* `direction` is the direction along which to perform the linesearch
* `eps` is the maximum allowed error in the resulting stepsize $t$
* `α` is the alpha parameter to backtracking linesearch
* `β` is the beta parameter to backtracking linesearch
* `maximum_iterations` is the maximum allowed number of iterations

**Problem 3.6** (5 pts)

Implement the *gradient descent* algorithm using the given linesearch method
```julia
function gradient_descent( f, x0, eps=1e-5, maximum_iterations=65536, linesearch_algorithm=exact_line_search )
    """
    f:                    the function to optimize It is called as "value, gradient = func( x, 1 )
    x0:                   the starting point
    eps:                  the maximum allowed error in the resulting stepsize t
    maximum_iterations:   the maximum allowed number of iterations
    linesearch_algorithm: the linesearch routine
    """
end
```

Test your method with above functions `f`, `boyd` and `quadratic` (with different types of matrix $H$)



